# Background

Interactive Brokers don't show the **Return on Margin (RoM)** and **Probability of the Return (PoR)** in its daily reports.

This program:  
&nbsp; a) Cleanses the daily activity report  
&nbsp; b) Gets **PoR** for each position  
&nbsp; c) Gets **RoM** from the Margin report and distributes it to the positions  
&nbsp; d) Prepares a "dashboard" googlespreadsheet 
&nbsp; e) Where possible, clubs the strategies for underlying based on a logic  

# Preparation

1. Log on to [ibkr.com](https://www.interactivebrokers.com.hk/en/home.php). Click on *Reports -> Activity -> Statements*
       Steps 2 to 9 to be done only once
2. Create a new **Customize Statements**. Name it as __*OptionsOnly*__.
3. Select tick boxes agains *Open Positions, Dividends* and *Option Exercises/Assignments* only. Leave all others blank
4. In **Profit and Loss:** - Choose *MTM and Realized P/L*
5. In **Statement Type:** - Choose *Activity Statement*
6. Choose your **Account:**
7. Choose **Activity Period:** as *Daily*
8. Choose **Format:** to be *CSV*
9. Save

## Run the IBKR report

10\. Choose *OptionsOnly* from Statement, **Format** as *CSV*  
11\. Run the report.  
12\. Rename the file to __*Yesterday.csv*__



# The Program

## a) Cleanse the daily activity report
Import the CSV and prepare the arrays

In [1]:
''' Program to extract stocks and options from IBKR Activity Statement
    Generated from ibkr.com -> Reports -> Activity Statements -> "Options Only"
    Choose previous day's report in CSV format

    Date: 18 June 2017     Rev: 1.0
'''

import csv
import pandas as pd

# for stock price history...
import pandas_datareader.data as web        
from datetime import datetime, timedelta

raw_data = []                      # raw_data list extracted from yesterday's csv file
clean_data = []                    # data cleansed from raw_data

Read the data.    
_**Yesterday.csv**_ is stored in a _datafiles_ subfolder under the current folder.
Filter out only _Open Positions_ without the _Totals_ to have base data-set

In [2]:
with open('datafiles/Yesterday.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',' )

    for row in reader:

        # filter for "Open Positions" without "Total"

        if (str(row[0]) == "Open Positions" and str(row[1]) != "Total" ):
            raw_data.append(row)    # Append raw_data list

show_df = pd.DataFrame(raw_data[0:9]) # Show the first 10 rows
show_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Open Positions,Header,DataDiscriminator,Asset Category,Currency,Symbol,Open,Quantity,Mult,Cost Price,Cost Basis,Close Price,Value,Unrealized P/L,Code
1,Open Positions,Data,Summary,Stocks,USD,AKAM,-,200,1,49.22696796,9845.393592,51.3,10260,414.606408,
2,Open Positions,Data,Lot,Stocks,USD,AKAM,"2017-05-12, 16:20:00",200,,49.22696796,9845.393592,51.3,10260,414.606408,
3,Open Positions,Data,Summary,Stocks,USD,APA,-,1900,1,50.607305856,96153.881126,46.08,87552,-8601.881126,SY
4,Open Positions,Data,Lot,Stocks,USD,APA,"2017-03-31, 16:20:00",1000,,50.90796408,50907.96408,46.08,46080,-4827.96408,
5,Open Positions,Data,Lot,Stocks,USD,APA,"2017-03-24, 16:20:00",300,,50.040461193,15012.138358,46.08,13824,-1188.138358,
6,Open Positions,Data,Lot,Stocks,USD,APA,"2017-03-24, 16:20:00",600,,50.389631147,30233.778688,46.08,27648,-2585.778688,
7,Open Positions,Header,DataDiscriminator,Asset Category,Currency,Symbol,Open,Quantity,Mult,Cost Price,Cost Basis,Close Price,Value,Unrealized P/L,Code
8,Open Positions,Data,Summary,Equity and Index Options,USD,AA 30JUN17 27.0 P,-,17,100,0.127931,217.4827,0.0274,46.58,-170.9027,


Replace the header, remove intermediate _DataDiscriminator_ rows and shorten _Asset Category_ to reflect _Options_

In [3]:
# Rename dataframe header with first element of raw_data
df = pd.DataFrame(raw_data)
df.columns = df.iloc[0]
df = df[1:]

# Remove excess DataDiscriminator rows
df = df[df.DataDiscriminator != "DataDiscriminator" ]

df['Asset Category'] = df['Asset Category'].str.replace('Equity and Index Options','Options') # Shorten "Options" in Category

show_df = pd.DataFrame(df[0:5]) # Show the first 6 rows
show_df

,Open Positions,Header,DataDiscriminator,Asset Category,Currency,Symbol,Open,Quantity,Mult,Cost Price,Cost Basis,Close Price,Value,Unrealized P/L,Code
1,Open Positions,Data,Summary,Stocks,USD,AKAM,-,200,1,49.22696796,9845.393592,51.3,10260,414.606408,
2,Open Positions,Data,Lot,Stocks,USD,AKAM,"2017-05-12, 16:20:00",200,,49.22696796,9845.393592,51.3,10260,414.606408,
3,Open Positions,Data,Summary,Stocks,USD,APA,-,1900,1,50.607305856,96153.881126,46.08,87552,-8601.881126,SY
4,Open Positions,Data,Lot,Stocks,USD,APA,"2017-03-31, 16:20:00",1000,,50.90796408,50907.96408,46.08,46080,-4827.96408,
5,Open Positions,Data,Lot,Stocks,USD,APA,"2017-03-24, 16:20:00",300,,50.040461193,15012.138358,46.08,13824,-1188.138358,


The _Open_ field has the _Transaction Date_ of the position.  
The _Open_ field in "Summary" rows have a '-' in it. It is replaced with the next row's _Open_ data.  
The next row is for _DataDiscriminator_ = "Lots", which also needs to be removed, after replacement.  
&nbsp; &nbsp; The _Open_ in the "Lots" row has time in it. This needs to be removed.  

In [4]:
# Replace "Open" column with date from the next row for "Summary"

df.loc[ df.DataDiscriminator=='Summary','Open'] = df.Open.shift(-1).str.slice(0,10)
df = df[ df.DataDiscriminator=='Summary' ]

df = df[df.DataDiscriminator == "Summary"] # Keep only the Summary rows in the dataset

Simplify the field names

In [5]:
# Simplify dataframe field names
df.columns.values[3] = 'Category'
df.columns.values[9] = 'Cost'
df.columns.values[11] = 'Close'

Split Options symbols to its own respective columns

In [6]:
# Assign and Split Options into separate columns
df = df.assign(**df.Symbol.str.split(' ', expand=True).rename(columns={0:'Symbol', 1:'Expiry', 2:'Strike', 3:'Right'}))

Stage the data with the most important fields

In [7]:
# Assemble the most important fields
clean_data = df[['Open', 'Symbol', 'Category', 'Right', 'Strike', 'Currency','Expiry', 'Quantity', 'Mult', 'Cost', 'Close']]

show_df = pd.DataFrame(clean_data[0:5]) # Show the first 6 rows
show_df

,Open,Symbol,Category,Right,Strike,Currency,Expiry,Quantity,Mult,Cost,Close
1,2017-05-12,AKAM,Stocks,None,None,USD,None,200,1,49.22696796,51.3
3,2017-03-31,APA,Stocks,None,None,USD,None,1900,1,50.607305856,46.08
8,2017-06-02,AA,Options,P,27.0,USD,30JUN17,17,100,0.127931,0.0274
11,2017-06-02,AA,Options,P,28.0,USD,30JUN17,-17,100,0.172045076,0.0645
14,2017-06-13,AABA,Options,C,60.0,USD,21JUL17,-7,100,0.34204137,0.1576


## b) Get the Probability of Return
The next step is to get the probability of return. We will do this by:
* Getting the underlying's historical price series
* Checking the standard deviation of the price for 45 days

In [8]:
#list out the 
Symbols = df.Symbol.unique()

In [12]:
end = datetime.today()
start = end - timedelta(days=5)        # days = days-to-subtract

price_hist = pd.DataFrame()

for symbol in Symbols:                  # needed for some suspect symbols like 'SHOP'
    try:
        f = web.DataReader(str(symbol), 'google', start, end)
    except:
        f = web.DataReader("NYSE:"+str(symbol), 'google', start, end)
    f['Symbol'] = symbol 
    price_hist = price_hist.append(f)

price_hist[:20]

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2017-06-21,48.78,49.34,48.43,49.28,1502896,AKAM
2017-06-22,49.19,50.61,48.97,50.02,2648922,AKAM
2017-06-23,50.12,51.51,49.92,51.30,6358993,AKAM
2017-06-21,47.35,47.43,45.26,45.66,5130304,APA
2017-06-22,45.87,46.25,45.15,45.63,2693344,APA
2017-06-23,45.72,46.19,45.52,46.08,10019488,APA
2017-06-21,30.59,30.59,29.63,29.69,3489616,AA
2017-06-22,29.81,30.25,29.66,29.80,2906662,AA
2017-06-23,30.13,31.43,29.79,31.09,6410938,AA
